# Algorytmy tekstowe - odległość edycyjna

### Implementacja algorytmu obliczania odległości edycyjnej oraz wizualizacji jego działania

In [1]:
import numpy as np
from enum import Enum
from collections import deque

class Operation(Enum):
    DELETE = 0
    INSERT = 1
    SUBSTITUTE = 2
    NO_CHANGE = 3

def delta(a, b):
    if a == b:
        return 0
    else:
        return 1

def edit_distance(a, b, delta_function):
    n, m = len(a), len(b)
    Edit = np.empty((n + 1, m + 1), dtype=np.int32)
    T = np.zeros((n + 1, m + 1), dtype=np.int32)

    for i in range(n + 1):
        T[i, 0], Edit[i, 0] = 0, i
    for j in range(m + 1):
        T[0, j], Edit[0, j] = 1, j

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            delta_value = delta_function(a[i - 1], b[j - 1])
            T[i, j], Edit[i, j] = min(enumerate([Edit[i - 1, j] + 1,
                                                 Edit[i, j - 1] + 1,
                                                 Edit[i - 1, j - 1] + delta_value]), key=lambda x: x[1])
            if T[i, j] == 2 and delta_value == 0:
                T[i, j] = 3


    return Edit, T

def edit_distance_visualization(a, b, edit_distance_function, delta_function):
    Edit, T = edit_distance_function(a, b, delta_function)


    queue = deque()
    i, j = len(a), len(b)
    print("Edit distance:", Edit[i, j])
    while i != 0 or j != 0:
        if T[i, j] == 0:
            queue.appendleft(Operation.DELETE)
            i -= 1
        elif T[i, j] == 1:
            queue.appendleft(Operation.INSERT)
            j -= 1
        else:
            if T[i, j] == 2: queue.appendleft(Operation.SUBSTITUTE)
            if T[i, j] == 3: queue.appendleft(Operation.NO_CHANGE)
            i -= 1
            j -= 1

    print("1st string:", a)
    i = j = 0
    current = a
    while len(queue) > 0:
        operation = queue.popleft()
        if operation == Operation.DELETE:
            print(f" * {current[:i]}(-{current[i]}){current[i+1:]}")
            current = current[:i] + current[i+1:]
            i -= 1
        elif operation == Operation.INSERT:
            print(f" * {current[:i]}(+{b[j]}){current[i:]}")
            current = current[:i] + b[j] + current[i:]
        elif operation == Operation.SUBSTITUTE:
            print(f" * {current[:i]}({current[i]}->{b[j]}){current[i+1:]}")
            current = current[:i] + b[j] + current[i+1:]

        i += 1
        j += 1

    print("2nd string:", b)

### Przedstawienie działania wizualizacji

#### 1. los - kloc

In [2]:
edit_distance_visualization("los", "kloc", edit_distance, delta)

Edit distance: 2
1st string: los
 * (+k)los
 * klo(s->c)
2nd string: kloc


#### 2. Łódź - Lodz

In [3]:
edit_distance_visualization("Łódź", "Lodz", edit_distance, delta)

Edit distance: 3
1st string: Łódź
 * (Ł->L)ódź
 * L(ó->o)dź
 * Lod(ź->z)
2nd string: Lodz


#### 3. kwintesencja - quintessence

In [4]:
edit_distance_visualization('kwintesencja', 'quintessence', edit_distance, delta)

Edit distance: 5
1st string: kwintesencja
 * (k->q)wintesencja
 * q(w->u)intesencja
 * quintes(+s)encja
 * quintessenc(j->e)a
 * quintessence(-a)
2nd string: quintessence


#### 4. ATGAATCTTACCGCCTCG - ATGAGGCTCTGGCCCCTG

In [5]:
edit_distance_visualization("ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG", edit_distance, delta)

Edit distance: 7
1st string: ATGAATCTTACCGCCTCG
 * ATGA(A->G)TCTTACCGCCTCG
 * ATGAG(T->G)CTTACCGCCTCG
 * ATGAGGCT(+C)TACCGCCTCG
 * ATGAGGCTCT(A->G)CCGCCTCG
 * ATGAGGCTCTG(+G)CCGCCTCG
 * ATGAGGCTCTGGCC(-G)CCTCG
 * ATGAGGCTCTGGCCCCT(-C)G
2nd string: ATGAGGCTCTGGCCCCTG


### Implementacja algorytmu obliczania najdłuższego wspólnego podciągu dla pary ciągów elementów

In [6]:
def lcs(a, b):
    n, m = len(a), len(b)
    LCS = np.zeros((n + 1, m + 1), dtype=np.int32)

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if a[i - 1] == b[j - 1]:
                LCS[i, j] = LCS[i - 1, j - 1] + 1
            else:
                LCS[i, j] = max(LCS[i, j - 1], LCS[i - 1, j])

    return LCS


def lcs_space_optimized(a, b):
    n, m = len(a), len(b)
    LCS = np.zeros((2, m + 1), dtype=np.int32)

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if a[i - 1] == b[j - 1]:
                LCS[1, j] = LCS[0, j - 1] + 1
            else:
                LCS[1, j] = max(LCS[1, j - 1], LCS[0, j])

        LCS[0, :] = LCS[1, :]

    return LCS[0, m]

### Podział tekstu na tokeny oraz stworzenie 2 nowych wersji, w których usunięto 3% losowych tokenów. Obliczenie długości najdłuższego podciągu wspólnych tokenów dla tych tekstów.

In [7]:
from spacy.lang.pl import Polish
from random import randint

In [8]:
def remove_random_tokens(tokens):
    new_tokens = []
    for token in tokens:
        if randint(1, 100) > 3:
            new_tokens.append(token.text_with_ws)

    return new_tokens

nlp = Polish()
tokenizer = nlp.tokenizer

with open("romeo-i-julia-700.txt", "r", encoding="utf-8") as f:
    text = f.read()
    tokens = tokenizer(text)

    new_tokens1 = remove_random_tokens(tokens)
    with open("text1.txt", "w", encoding="utf-8") as f1:
        for token in new_tokens1:
            f1.write(token)

    new_tokens2 = remove_random_tokens(tokens)
    with open("text2.txt", "w", encoding="utf-8") as f2:
        for token in new_tokens2:
            f2.write(token)

    print("Liczba tokenów w 1 wersji tekstu:", len(new_tokens1))
    print("Liczba tokenów w 2 wersji tekstu:", len(new_tokens2))
    print("Długość najdłuższego podciągu wspólnych tokenów:", lcs_space_optimized(new_tokens1, new_tokens2))

Liczba tokenów w 1 wersji tekstu: 2614
Liczba tokenów w 2 wersji tekstu: 2616
Długość najdłuższego podciągu wspólnych tokenów: 2533


### Implementacja narzędzia podobnego do diff

In [9]:
def diff(a, b):
    LCS = lcs(a, b)

    lines = []
    i, j = len(a), len(b)
    while i != 0 or j != 0:
        if i > 0 and j > 0 and a[i - 1] == b[j - 1]:
            i -= 1
            j -= 1
        elif j > 0 and (i == 0 or LCS[i, j - 1] >= LCS[i - 1, j]):
            j -= 1
            lines.append((1, j))
        else:
            i -= 1
            lines.append((0, i))

    for i in range(len(lines) - 1, -1, -1):
        file, line = lines[i]
        if file == 0:
            print(f"> [{line}]", a[line], end="")
        else:
            print(f"< [{line}]", b[line], end="")

        if i > 0 and lines[i][0] == 1 and lines[i - 1][0] == 0:
            print()

### Przedstawienie działania narzędzia dla przygotowanych tekstów

In [10]:
with open("text1.txt", "r", encoding="utf-8") as f1, open("text2.txt", "r", encoding="utf-8") as f2:
    diff(f1.readlines(), f2.readlines())

> [5] ISBN 978-83-288--9
< [5] ISBN 978-83288-2903-9

> [9] OSOBY:
> [10]  * — książę panujący w Weronie
< [9] OSOBY:* ESKALUS — książę panujący w Weronie

> [12]  * MONTEKI, KAPULET — naczelnicy dwóch domów nieprzyjaznych sobie
> [13]  * STARZEC — stryjeczny brat Kapuleta
> [14]  * ROMEO — syn Montekiego
> [15]  * MERKUCJO — krewny księcia
> [16]  * BENWOLIO — synowiec Montekiego
> [17]  * TYBALT — krewny Pani Kapulet
< [11]  * , KAPULET — naczelnicy dwóch domów nieprzyjaznych sobie
< [12]  * — stryjeczny brat Kapuleta
< [13]  * ROMEO — syn 
< [14]  * MERKUCJO krewny księcia
< [15]  * — synowiec 
< [16]  * TYBALT krewny Pani Kapulet

> [28]  * MONTEKI — małżonka Montekiego
< [27]  * PANI MONTEKI — małżonka Montekiego

> [30]  * JULIA — córka 
< [29]  * JULIA — córka Kapuletów

> [32]  * Obywatele weroneńscy, różne osoby obojej, liczący się do przyjaciół domów, maski, straż wojskowa i inne osoby.
< [31]  * Obywatele weroneńscy, różne osoby płci obojej, liczący się do przyjaciół obu dom